## Deliverable 3. Create a Travel Itinerary Map.

In [10]:

# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

import numpy as np

In [11]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.

vacation_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_df.head()

,Hotel_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,3,Puerto Ayora,EC,71.78,-0.7393,-90.3518,Finch Bay Galapagos Hotel
1,8,Jamestown,US,71.24,42.0970,-79.2353,Hampton Inn & Suites Jamestown
2,9,Nea Stira,GR,77.18,38.1833,24.2000,Venus Beach Hotel
3,10,Arraial Do Cabo,BR,71.56,-22.9661,-42.0278,Pousada Porto Praia
4,11,Kapaa,US,79.12,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [14]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [15]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [79]:
colombia_cities = vacation_df.loc[(vacation_df["Country"] == "CO") | (vacation_df["Country"] == "VE" )]
colombia_cities

,Hotel_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
14,53,San Cristobal,VE,73.58,7.7669,-72.2250,Pirineos
29,179,Inirida,CO,72.59,3.8653,-67.9239,Fuente del Guania Hotel de Lujo
68,350,Purificacion,CO,72.55,3.8587,-74.9313,Hotel Villa de las Palmas
74,371,Mapiripan,CO,72.48,2.8892,-72.1256,Monserrate
94,461,Mani,CO,71.76,4.8172,-72.2886,Hotel Belgrado


In [91]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = colombia_cities.loc[(colombia_cities["City"] == "Purificacion")]
vacation_end = colombia_cities.loc[(colombia_cities["City"] == "Purificacion")]
vacation_stop1 = colombia_cities.loc[(colombia_cities["City"] == "Mani")]
vacation_stop2 = colombia_cities.loc[(colombia_cities["City"] == "San Cristobal")] 
vacation_stop3 =  colombia_cities.loc[(colombia_cities["City"] == "Mapiripan")]
vacation_stop4 = colombia_cities.loc[(colombia_cities["City"]=="Inirida")]


In [92]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start1 = vacation_start["Lat"].to_numpy()
start2 = vacation_start["Lng"].to_numpy()
start = np.concatenate((start1,start2))
end = start

stop1a = vacation_stop1["Lat"].to_numpy()
stop1b = vacation_stop1["Lng"].to_numpy()
stop1 = np.concatenate((stop1a,stop1b))

stop2a = vacation_stop2["Lat"].to_numpy()
stop2b = vacation_stop2["Lng"].to_numpy()
stop2 = np.concatenate((stop2a,stop2b))

stop3a = vacation_stop3["Lat"].to_numpy()
stop3b = vacation_stop3["Lng"].to_numpy()
stop3 = np.concatenate((stop3a,stop3b))

In [86]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
import gmaps
import gmaps.datasets

fig = gmaps.figure()

starttostop1 = gmaps.directions_layer(start, stop1, travel_mode='WALKING')
fig.add_layer(starttostop1)

stop1to2 = gmaps.directions_layer(stop1, stop2, travel_mode='WALKING')
fig.add_layer(stop1to2)

stop2toend = gmaps.directions_layer(stop2, end, travel_mode='WALKING')
fig.add_layer(stop2toend)

stop3 = gmaps.directions_layer(stop2, stop3, travel_mode='WALKING')
fig.add_layer(stop3)

fig


Figure(layout=FigureLayout(height='420px'))

In [94]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_stop4],ignore_index=True)
itinerary_df

,Hotel_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,350,Purificacion,CO,72.55,3.8587,-74.9313,Hotel Villa de las Palmas
1,461,Mani,CO,71.76,4.8172,-72.2886,Hotel Belgrado
2,53,San Cristobal,VE,73.58,7.7669,-72.2250,Pirineos
3,371,Mapiripan,CO,72.48,2.8892,-72.1256,Monserrate
4,179,Inirida,CO,72.59,3.8653,-67.9239,Fuente del Guania Hotel de Lujo


In [96]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [97]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))